In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

#returns a pandas dataframe with the data from a specified csv file
def loadData(filename):
    data = pd.read_csv(filename)
    return data

#returns one of 13 chess opening subtypes given a specific opening
def openType(openStr):
    open = openStr[0]
    num = int(openStr[1:])
    if (openStr[0] == 'A'):
        if (num < 40):
            open += '00'
        elif (num < 45):
            open += '40'
        elif (num < 50):
            open += '45'
        elif (num < 80):
            open += '50'
        else:
            open += '80'
    if (openStr[0] == 'B'):
        if (num < 20):
            open += '00'
        else:
            open += '20'
    if (openStr[0] == 'C'):
        if (num < 20):
            open += '00'
        else:
            open += '20'
    if (openStr[0] == 'D'):
        if (num < 70):
            open += '00'
        else:
            open += '70'
    if (openStr[0] == 'E'):
        if (num < 60):
            open += '00'
        else:
            open += '60'
    return open

#returns a dataset and labelset from a pandas dataframe
#the returned dataset includes the first (openingPly) moves preformed
#the returned labelset is the opening subtype defined by the moves preformed in the dataset
def dataExtraction(baseData):
    data = []
    labels = []
    
    for i in range(len(baseData.index)):
        row = baseData.iloc[i, :]
        opening = []
        moveText = row[12].split()
        for j in range(row[15]):
            opening.append(moveText[j])
        data.append(opening)
        labels.append(openType(row[13]))
    
    return data, labels

#returns a numericized version of the input array along with the dictionary used for conversion
def numericize(textData):
    numericDic = {}
    conversion = []
    if type(textData[0]) != list:
        for label in textData:
            if label not in numericDic:
                numericDic[label] = len(numericDic) + 1
            conversion.append(numericDic[label])
    else:
        for text in textData:
            moveList = []
            for move in text:
                if move not in numericDic:
                    numericDic[move] = len(numericDic) + 1
                moveList.append(numericDic[move])
            conversion.append(moveList)
            
    return numericDic, conversion

In [4]:
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

data = loadData('games.csv')
maxMoves = max(data.iloc[:,15])
moves, labels = dataExtraction(data)
moveDic, numMoves = numericize(moves)
labelDic, numLabels = numericize(labels)

numMoves = sequence.pad_sequences(numMoves, maxlen=maxMoves)

#builds subsets for testing and training data at an 80-20 split
X_train, X_test, y_train, y_test = train_test_split(numMoves, numLabels, test_size=0.2, stratify=numLabels)

In [5]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 10 #64
num_epochs = 2 #3
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

NameError: name 'vocabulary_size' is not defined

In [5]:
if "this" in "whatisthis":
    print("true")
print(openType("C14"))

true
C00
